In [1]:
import numpy as np
import pandas as pd

In [6]:
# Load data point, weights, and biases from CSV files
data_point = np.array([-1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1])
target_label = np.array([3])


w0_input_hidden1 = pd.read_csv(
    r'Task_1\a\w.csv', index_col=0, nrows=14, header=None).values
w0_hidden1_hidden2 = pd.read_csv(r'Task_1\a\w.csv', index_col=0, skiprows=range(14), nrows=100, usecols=range(41), header=None).values
w0_hidden2_output = pd.read_csv(r'Task_1\a\w.csv', index_col=0, skiprows=range(114), nrows=40, usecols=range(5), header=None).values

b0_hidden1 = pd.read_csv(
    r'Task_1\a\b.csv', index_col=0, nrows=1, header=None).values
b0_hidden2 = pd.read_csv(r'Task_1\a\b.csv', index_col=0, skiprows=range(1), nrows=1, usecols=range(41), header=None).values
b0_output = pd.read_csv(r'Task_1\a\b.csv', index_col=0, skiprows=range(2), nrows=1, usecols=range(5), header=None).values

w1_input_hidden1 = pd.read_csv(
    r'Task_1\b\w-100-40-4.csv', index_col=0, nrows=14, header=None).values
w1_hidden1_hidden2 = pd.read_csv(r'Task_1\b\w-100-40-4.csv', index_col=0, skiprows=range(14), nrows=100, usecols=range(41), header=None).values
w1_hidden2_output = pd.read_csv(r'Task_1\b\w-100-40-4.csv', index_col=0, skiprows=range(114), nrows=40, usecols=range(5), header=None).values

b1_hidden1 = pd.read_csv(
    r'Task_1\b\b-100-40-4.csv', index_col=0, nrows=1, header=None).values
b1_hidden2 = pd.read_csv(r'Task_1\b\b-100-40-4.csv', index_col=0, skiprows=range(1), nrows=1, usecols=range(41), header=None).values
b1_output = pd.read_csv(r'Task_1\b\b-100-40-4.csv', index_col=0, skiprows=range(2), nrows=1, usecols=range(5), header=None).values

correct_grad_w0_input_hidden1 = pd.read_csv(
    r'Task_1\a\true-dw.csv', nrows=14, header=None).values
correct_grad_w0_hidden1_hidden2 = pd.read_csv(r'Task_1\a\true-dw.csv', skiprows=range(14), nrows=100, usecols=range(40), header=None).values
correct_grad_w0_hidden2_output = pd.read_csv(r'Task_1\a\true-dw.csv', skiprows=range(114), nrows=40, usecols=range(4), header=None).values

correct_grad_b0_hidden1 = pd.read_csv(
    r'Task_1\a\true-db.csv', nrows=1, header=None).values
correct_grad_b0_hidden2 = pd.read_csv(r'Task_1\a\true-db.csv', skiprows=range(1), nrows=1, usecols=range(40), header=None).values
correct_grad_b0_output = pd.read_csv(r'Task_1\a\true-db.csv', skiprows=range(2), nrows=1, usecols=range(4), header=None).values


In [3]:
# Define ReLU activation function and its derivative
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Define Softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Cross-entropy loss function
def cross_entropy_loss(y_true, new_y_pred):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(new_y_pred + 1e-8)) / m

# One-hot encoding for labels
def one_hot_encode(y, num_classes):
    one_hot = np.zeros((y.size, num_classes))
    one_hot[np.arange(y.size), y] = 1
    return one_hot

In [11]:
one_hot_encode(target_label,4)

array([[0., 0., 0., 1.]])

In [9]:
relu(4)

4

In [7]:
new_z1 = np.dot(data_point, w0_input_hidden1) + b0_hidden1
new_a1 = relu(new_z1)

new_z2 = np.dot(new_a1, w0_hidden1_hidden2) + b0_hidden2
a2 = relu(new_z2)

new_z3 = np.dot(a2, w0_hidden2_output) + b0_output
new_a3 = softmax(new_z3)

new_a3

array([[1.81297987e-158, 5.20887762e-137, 1.00000000e+000,
        2.00080740e-085]])

In [8]:
new_y_pred = new_a3
y_true = one_hot_encode(target_label,4)

cel = cross_entropy_loss(y_true, new_y_pred)
cel

18.420680743952367

In [9]:
dz3 = y_pred - y_true
dw3 = np.dot(a2.T, dz3)
db3 = np.sum(dz3, axis=0, keepdims=True)

dz2 = np.dot(dz3, w0_hidden2_output.T) * relu_derivative(a2)
dw2 = np.dot(a1.T, dz2)
db2 = np.sum(dz2, axis=0, keepdims=True)

dz1 = np.dot(dz2, w0_hidden1_hidden2.T) * relu_derivative(a1)
dw1 = np.dot(data_point.reshape(-1,1), dz1)

In [10]:
np.allclose(dw1, correct_grad_w0_input_hidden1, atol=1e-8), np.allclose(dw2, correct_grad_w0_hidden1_hidden2, atol=1e-8), np.allclose(dw3, correct_grad_w0_hidden2_output, atol=1e-8), np.allclose(db1, correct_grad_b0_hidden1, atol=1e-8), np.allclose(db2, correct_grad_b0_hidden2, atol=1e-8), np.allclose(db3, correct_grad_b0_output, atol=1e-8)

(False, False, False, False, False, False)

## new

In [5]:
z1 = np.dot(data_point, w1_input_hidden1) + b1_hidden1
a1 = relu(z1)

z2 = np.dot(a1, w1_hidden1_hidden2) + b1_hidden2
a2 = relu(z2)

z3 = np.dot(a2, w1_hidden2_output) + b1_output
a3 = softmax(z3)

y_pred = a3
y_true = one_hot_encode(target_label,4)

dz3 = y_pred - y_true
dw3 = np.dot(a2.T, dz3)
db3 = np.sum(dz3, axis=0, keepdims=True)

dz2 = np.dot(dz3, w0_hidden2_output.T) * relu_derivative(a2)
dw2 = np.dot(a1.T, dz2)
db2 = np.sum(dz2, axis=0, keepdims=True)

dz1 = np.dot(dz2, w0_hidden1_hidden2.T) * relu_derivative(a1)
dw1 = np.dot(data_point.reshape(-1,1), dz1)
db1 = np.sum(dz1, axis=0, keepdims=True)

# save the gradients of weights and biases to specific csv files
dw = pd.DataFrame(dw1)
dw.to_csv(r'submission2\true-dw.csv', header=False, index=False)

# save the gradients of weights and biases to specific csv files
dw = pd.DataFrame(dw2)
dw.to_csv(r'submission2\true-dw.csv', header=False, index=False, mode='a')

# save the gradients of weights and biases to specific csv files
dw = pd.DataFrame(dw3)
dw.to_csv(r'submission2\true-dw.csv', header=False, index=False, mode='a')

db = pd.DataFrame(db1)
db.to_csv(r'submission2\true-db.csv', header=False, index=False)

db = pd.DataFrame(db2)
db.to_csv(r'submission2\true-db.csv', header=False, index=False, mode='a')

db = pd.DataFrame(db3)
db.to_csv(r'submission2\true-db.csv', header=False, index=False, mode='a')


In [40]:
# save the gradients of weights and biases to specific csv files
dw = pd.DataFrame(dw1)
dw.to_csv(r'submission\true-dw.csv', header=False, index=False)

In [41]:
# save the gradients of weights and biases to specific csv files
dw = pd.DataFrame(dw2)
dw.to_csv(r'submission\true-dw.csv', header=False, index=False, mode='a')

In [42]:
# save the gradients of weights and biases to specific csv files
dw = pd.DataFrame(dw3)
dw.to_csv(r'submission\true-dw.csv', header=False, index=False, mode='a')

In [43]:
db = pd.DataFrame(db1)
db.to_csv(r'submission\true-db.csv', header=False, index=False)

db = pd.DataFrame(db2)
db.to_csv(r'submission\true-db.csv', header=False, index=False, mode='a')

db = pd.DataFrame(db3)
db.to_csv(r'submission\true-db.csv', header=False, index=False, mode='a')